In [1]:

import os


In [2]:
%pwd

'/workspaces/Ogrenci_Performans_Degerlendirmesi/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/workspaces/Ogrenci_Performans_Degerlendirmesi'

In [5]:
from dataclasses import dataclass


In [6]:
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
!export PYTHONPATH=$pwd

In [8]:
from importlib import reload
from src.Ogrenci_Performansı.utils import common
reload(common)


<module 'src.Ogrenci_Performansı.utils.common' from '/workspaces/Ogrenci_Performans_Degerlendirmesi/src/Ogrenci_Performansı/utils/common.py'>

In [9]:
from src.Ogrenci_Performansı.constants import *
from src.Ogrenci_Performansı.utils.common import read_yaml, create_directories 


In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      
          




    

In [11]:
import os
import zipfile
import urllib.request as request
from src.Ogrenci_Performansı import logger
from src.Ogrenci_Performansı.utils.common import get_size



In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager() #benim değişkenlerimin değerlerini tutuyor.
    data_ingestion_config = config.get_data_ingestion_config() # bu methodla config.yaml içindeki bütün değerleri çağırıyorum
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file() #görüntüleri indirir ve zip uzantılı olarak verilen dizine atar
    data_ingestion.extract_zip_file() # zipten çıkarır ve verilen dosya dizinde kayıt eder
except Exception as e:
    raise e

[2024-11-07 20:31:49,963: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Expires: Thu, 07 Nov 2024 20:31:49 GMT
Date: Thu, 07 Nov 2024 20:31:49 GMT
Cache-Control: private, max-age=0
Last-Modified: Thu, 07 Mar 2024 06:14:12 GMT
ETag: "47d2ff7dbfc7997cc2cf67cb80e41cdd"
x-goog-generation: 1709792052142033
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 8907
Content-Type: application/zip
x-goog-hash: crc32c=A49eQg==
x-goog-hash: md5=R9L/fb/HmXzCz2fLgOQc3Q==
x-goog-storage-class: MULTI_REGIONAL
Accept-Ranges: bytes
Content-Length: 8907
Access-Control-Allow-Origin: *
Access-Control-Expose-Headers: Content-Type
Access-Control-Expose-Headers: User-Agent
X-GUploader-UploadID: AHmUCY2K4rO2t_9fIdfURAi9zBZmPBEGr9M3UFIJpl50IbKDNon6JHqvOYyiJ4wzli6nJmTTyrs
Server: UploadServer
Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000
Connection: close

 
